In [10]:
import time
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "HuggingFaceTB/SmolLM-1.7B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id, torch_dtype=torch.bfloat16, device_map="auto"
)



tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.85G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [14]:
import random

prompts = ["Summarize the history of AI."] * 5000  # adjust for batching

# Word pools
adjectives = [
    "ancient", "futuristic", "mysterious", "brilliant", "dark", "colorful",
    "peaceful", "chaotic", "lonely", "vibrant", "silent", "melancholic",
    "red", "blue", "golden", "silver", "emerald", "gentle", "stormy", "radiant"
]

nouns = [
    "city", "forest", "ocean", "planet", "dream", "creature", "robot",
    "painting", "poem", "machine", "castle", "storm", "garden", "ship",
    "dimension", "universe", "song", "memory", "light", "shadow"
]

actions = [
    "describe", "explain", "analyze", "summarize", "imagine", "narrate",
    "predict", "compare", "design", "debate", "reimagine", "illustrate",
    "explore", "evaluate", "invent", "critique", "compose", "argue", "translate", "simulate"
]


prompts = []
for _ in range(500):
    adj = random.choice(adjectives)
    noun = random.choice(nouns)
    action = random.choice(actions)

    prompt_type = random.choice([
        f"{action.capitalize()} a {adj} {noun}.",
        f"Write a story about a {adj} {noun}.",
        f"What would happen if a {adj} {noun} could think?",
        f"Give three facts about a {adj} {noun}.",
        f"Explain the significance of a {adj} {noun} in society.",
        f"Imagine discovering a {adj} {noun} — describe it in detail."
    ])

    prompts.append(prompt_type)



if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer(prompts, return_tensors="pt", padding=True).to("cuda")


start = time.time()
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=100)
elapsed = time.time() - start

print(f"Batch size: {len(prompts)} | Time: {elapsed:.2f}s")


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


OutOfMemoryError: CUDA out of memory. Tried to allocate 62.00 MiB. GPU 0 has a total capacity of 39.56 GiB of which 32.88 MiB is free. Process 12265 has 39.52 GiB memory in use. Of the allocated memory 38.19 GiB is allocated by PyTorch, and 845.99 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [19]:
import gc
gc.collect()

0

In [18]:
torch.cuda.empty_cache()

In [17]:
del model
del inputs
del outputs